In [9]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [222]:
import numpy as np
import cv2

N = 224
random_phase = np.random.rand(4*N, 4*N) * 2 * np.pi
def create_speckle(phase, lam=0.532e-6, f1=0.20, f2=0.10):
    N   = phase.shape[0]
    Np  = 4 * N                    # 패딩된 격자 크기 # 4 * 224 = 896
    pad = (Np - N)//2              # (896 - 224)//2 = 336
    
    # 1) 입력면
    M = np.exp(1j * phase)
    M = np.pad(M, pad_width=pad, mode='constant', constant_values=0)
    
    # 2) 푸리에면 무작위 위상 디퓨저
    D = np.exp(1j * random_phase)          # 위상만 가진 디퓨저
    
    # 3) 1차 FFT (푸리에면)
    F = np.fft.fftshift(np.fft.fft2(M)) * D
    
    # 4) 2차 FFT (영상면)
    U = np.fft.ifft2(np.fft.ifftshift(F))
    
    # 배율 보정 & 뒤집기
    U *= (1/(lam**2 * f1 * f2))
    U = np.fliplr(U)
    
    # 5) 결과 크롭 (원본보다 2배 넓게 보기)
    out_size = 2*N           # 예: 2N×2N ROI # 2*224=448
    start = (Np - out_size)//2
    I = np.abs(U[start:start+out_size, start:start+out_size])**2
    
    # 필요하면 다시 0.5배 축소(실제 배율) 후 붙여넣기
    I_small = cv2.resize(I, dsize=(N, N), interpolation=cv2.INTER_AREA)
    I_small = I_small / np.max(I_small[N//4:3*N//4, N//4:3*N//4])
    return I_small


In [226]:
import os
from tqdm import tqdm

path = 'dataset/test/phase'
vmax =  1.3855439642564844
for file in tqdm(os.listdir(path)):
    phase = Image.open(os.path.join(path, file)).convert('L')
    phase = np.array(phase)
    phase = phase / 255 * 2 * np.pi
    phase = cv2.resize(phase, dsize=(224, 224))
    speckle = create_speckle(phase)
    speckle = speckle / vmax * 225
    Image.fromarray(speckle.astype('uint8')).save(f'dataset/test/speckle/{file}')

100%|██████████| 200/200 [00:15<00:00, 13.04it/s]


In [224]:
import os
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# create_speckle 함수가 정의되어 있다고 가정합니다.
# 예시: def create_speckle(phase): ... return speckle_pattern

# --- 경로 설정 ---
path = 'dataset/train/phase'

all_speckle_values = []

print("1단계: 전체 데이터셋의 99.9 퍼센타일 값을 계산합니다...")
for file in tqdm(os.listdir(path)):
    # 이미지를 불러오고 phase 맵으로 변환
    phase = Image.open(os.path.join(path, file)).convert('L')
    phase = np.array(phase)
    phase = phase / 255.0 * 2 * np.pi
    phase = cv2.resize(phase, dsize=(224, 224))

    # 스페클 생성 (이 함수는 이미 정의되어 있어야 합니다)
    speckle = create_speckle(phase)

    # 생성된 스페클 값들을 하나의 리스트에 모두 추가
    all_speckle_values.extend(speckle.flatten())

# 모든 스페클 값들 중에서 99.9 퍼센타일 계산
# 데이터셋이 매우 크면 이 부분에서 메모리를 많이 사용할 수 있습니다.
vmax_p999 = np.percentile(all_speckle_values, 99.99)

print(f"\n계산된 99.9 퍼센타일 (vmax): {vmax_p999}")

# 이제 이 vmax_p999 값을 2단계에서 사용합니다.

1단계: 전체 데이터셋의 99.9 퍼센타일 값을 계산합니다...


100%|██████████| 100/100 [00:07<00:00, 12.86it/s]



계산된 99.9 퍼센타일 (vmax): 1.3855439642564844


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from pytorch_msssim import ssim, ms_ssim # <--- 이 부분을 추가하세요
import torch.nn.functional as F

# --- 기본 설정 (이전과 동일) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
N = 224
LEARNING_RATE_MODEL = 1e-4
LEARNING_RATE_PHASE = 1e-3

import torch, math
import torch.nn as nn

# ───────────────── ① 고정 pupil 생성 ─────────────────
def make_pupil(N, radius_px=10, y_shift_px=-10):
    x = torch.arange(-N//2, N//2)
    X, Y = torch.meshgrid(x, x, indexing='ij')
    P = (((X+y_shift_px)**2 + Y**2) < radius_px**2).float()
    return P          # (N,N) real, 0/1

import torch
import torch.nn as nn
import torch.nn.functional as F


# ─────────────────── Residual DoubleConv ───────────────────
class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
        )
        self.act = nn.ReLU(inplace=True)
        # 입력·출력 채널이 다르면 1×1 프로젝션
        self.skip = nn.Identity() if in_ch == out_ch else nn.Conv2d(in_ch, out_ch, 1, bias=False)

    def forward(self, x):
        return self.act(self.conv(x) + self.skip(x))

# ─────────────────── Channel Attention (Squeeze & Excite) ───────────────────
class SE(nn.Module):
    def __init__(self, ch, r=8):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc   = nn.Sequential(nn.Linear(ch, ch//r, bias=False),
                                  nn.ReLU(inplace=True),
                                  nn.Linear(ch//r, ch, bias=False),
                                  nn.Sigmoid())
    def forward(self, x):
        b,c,_,_ = x.shape
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# ─────────────────── Deep U-Net ───────────────────
class DeepUNet(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, base=32):
        super().__init__()
        ch = base
        # ─ Encoder (5 스테이지) ─
        self.e1 = ResBlock(in_ch,  ch)          # 224×224
        self.e2 = ResBlock(ch,     ch*2)        # 112×112
        self.e3 = ResBlock(ch*2,   ch*4)        # 56×56
        self.e4 = ResBlock(ch*4,   ch*8)        # 28×28
        self.e5 = ResBlock(ch*8,   ch*16)       # 14×14

        self.pool = nn.MaxPool2d(2)

        # ─ Bottleneck (7×7) ─
        self.bottleneck = ResBlock(ch*16, ch*32)

        # ─ Decoder ─
        self.up5 = nn.ConvTranspose2d(ch*32, ch*16, 2, 2)  # 14×14
        self.d5  = ResBlock(ch*32, ch*16)
        self.se5 = SE(ch*16)

        self.up4 = nn.ConvTranspose2d(ch*16, ch*8, 2, 2)   # 28×28
        self.d4  = ResBlock(ch*16, ch*8)
        self.se4 = SE(ch*8)

        self.up3 = nn.ConvTranspose2d(ch*8, ch*4, 2, 2)    # 56×56
        self.d3  = ResBlock(ch*8, ch*4)
        self.se3 = SE(ch*4)

        self.up2 = nn.ConvTranspose2d(ch*4, ch*2, 2, 2)    # 112×112
        self.d2  = ResBlock(ch*4, ch*2)
        self.se2 = SE(ch*2)

        self.up1 = nn.ConvTranspose2d(ch*2, ch, 2, 2)      # 224×224
        self.d1  = ResBlock(ch*2, ch)
        self.se1 = SE(ch)

        # ─ Output ─
        self.out_conv = nn.Conv2d(ch, out_ch, 1)
        self.act = nn.Sigmoid()

    def forward(self, x):
        # Encoder
        e1 = self.e1(x)
        e2 = self.e2(self.pool(e1))
        e3 = self.e3(self.pool(e2))
        e4 = self.e4(self.pool(e3))
        e5 = self.e5(self.pool(e4))

        b  = self.bottleneck(self.pool(e5))

        # Decoder with SE attention
        d5 = self.se5(self.d5(torch.cat([self.up5(b), e5], dim=1)))
        d4 = self.se4(self.d4(torch.cat([self.up4(d5), e4], dim=1)))
        d3 = self.se3(self.d3(torch.cat([self.up3(d4), e3], dim=1)))
        d2 = self.se2(self.d2(torch.cat([self.up2(d3), e2], dim=1)))
        d1 = self.se1(self.d1(torch.cat([self.up1(d2), e1], dim=1)))

        return self.act(self.out_conv(d1))


class DeepDiffuser(nn.Module):
    """
    CNN을 사용하여 Diffuser의 복잡한 물리적 특성을 모델링합니다.
    이 네트워크는 최종적으로 복소수 필드(Complex Field) H를 생성합니다.
    """
    def __init__(self, N=224, latent_channels=16):
        super().__init__()
        # 이 네트워크는 고정된 패턴을 생성하므로, 입력 대신 학습 가능한 파라미터로 시작합니다.
        # 이 `initial_grid`가 학습을 통해 복잡한 패턴으로 발전합니다.
        self.initial_grid = nn.Parameter(torch.randn(1, latent_channels, N, N))

        self.net = nn.Sequential(
            nn.Conv2d(latent_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # 최종 출력은 복소수의 실수부(real)와 허수부(imag) 2개의 채널입니다.
            nn.Conv2d(64, 2, kernel_size=1) 
        )

    def forward(self):
        # 네트워크를 통과시켜 2채널 맵(실수부, 허수부)을 얻습니다.
        complex_parts = self.net(self.initial_grid)
        
        real_part = complex_parts[:, 0, :, :]
        imag_part = complex_parts[:, 1, :, :]
        
        # 두 채널을 합쳐 복소수 텐서를 만듭니다.
        H = torch.complex(real_part, imag_part)
        
        # (선택 사항) 안정적인 학습을 위해 출력의 크기를 제한할 수 있습니다.
        # 예: 진폭이 1을 넘지 않도록 정규화
        H = H / (torch.max(torch.abs(H)) + 1e-8)
        
        return H.squeeze(0) # 배치 차원 제거 (N, N)

class SpeckleSimulator(nn.Module):
    def __init__(self, N=224, lam=0.532e-6, f1=0.2, f2=0.1,
                 radius_px=10, y_shift_px=-10):
        super().__init__()
        
        self.N = N
        self.lam = lam
        self.f1 = f1
        self.f2 = f2
        
        # 시스템 배율 계산
        magnification = self.f2 / self.f1
        
        self.sim_size = N * 4   # 4*224 = 896
        self.pad_size = (self.sim_size - N) // 2   # (896 - 224)//2 = 336
        
        # self.radius_px = radius_px
        # self.y_shift_px = y_shift_px
        
        diffuser_phase = torch.rand(self.sim_size, self.sim_size) * 2 * torch.pi
        self.diffuser = nn.Parameter(diffuser_phase, requires_grad=True)
        
        self.diffuser_net = DeepDiffuser(N=4*N)
        # pupil_mask = make_pupil(N, radius_px, y_shift_px)
        # self.register_buffer('P', pupil_mask)
        
    def forward(self, phase):
        D = torch.exp(1j * self.diffuser)
        # H = D * self.P
        # H = D
        H = self.diffuser_net()
        M = torch.exp(1j * phase)
        M = F.pad(M, pad=(self.pad_size, self.pad_size, self.pad_size, self.pad_size), 
                               mode='constant', value=0)
        
        f = torch.fft.fftshift(torch.fft.fft2(M)) * H.cuda()
        U = torch.fft.ifft2(torch.fft.ifftshift(f))
        U = U / (self.lam**2 * self.f1 * self.f2)
        I = torch.abs(U)**2
        I = I / torch.max(I).item()
        I = torch.flip(I, dims=[-1])
        
        out_size = 2 * self.N   # 2*224 = 448
        start = (self.sim_size - out_size) // 2
        result = I[..., start:start+out_size, start:start+out_size]
        result = F.interpolate(result, size=(self.N, self.N), mode='area')
        
        return result


# --- 🌟 데이터셋 클래스 정의 🌟 ---
class SpeckleDataset(Dataset):
    def __init__(self, phase_dir, speckle_dir):
        # 이미지 파일 경로 리스트 가져오기
        self.phase_paths = sorted(os.listdir(phase_dir))
        self.speckle_paths = sorted(os.listdir(speckle_dir))

        self.phase_list = []
        self.speckle_list = []

        # 각 이미지에 대한 위상을 저장할 딕셔너리
        for path in self.phase_paths:
            phase = Image.open(os.path.join(phase_dir, path)).convert('L')
            phase = np.array(phase)
            phase = phase / 255 * 2 * np.pi
            phase = cv2.resize(phase, dsize=(N, N))
            self.phase_list.append(phase)
        
        for path in self.speckle_paths:
            speckle = Image.open(os.path.join(speckle_dir, path)).convert('L')
            speckle = np.array(speckle)
            speckle = speckle / 255
            speckle = cv2.resize(speckle, dsize=(N, N))
            self.speckle_list.append(speckle)
            
    def __len__(self):
        return len(self.phase_list)
    
    def __getitem__(self, idx):
        phase = self.phase_list[idx]
        speckle = self.speckle_list[idx]
        phase = torch.from_numpy(phase)
        speckle = torch.from_numpy(speckle)
        return phase, speckle
    
train_dataset = SpeckleDataset('dataset/train/phase', 'dataset/train/speckle')
test_dataset = SpeckleDataset('dataset/test/phase', 'dataset/test/speckle')
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

model = model = SpeckleSimulator().cuda()
# model = DeepUNet(in_ch=1, out_ch=1, base=32).cuda()
model = model.cuda()

loss_fn = torch.nn.L1Loss()
s = torch.tensor(1.0, requires_grad=True)
optimizer = torch.optim.Adam(list(model.parameters()) + [s], lr=5e-3)

Using device: cuda


In [5]:
optimizer = torch.optim.Adam(list(model.parameters()) + [s], lr=5e-4)

In [6]:
num_epoch = 500
for i in range(num_epoch):
    print(f'========= Epoch : {i} =========')
    model = model.train()
    train_loss = []
    for phase, speckle in train_loader:
        optimizer.zero_grad()
        phase = phase.type(torch.float32).cuda()
        speckle = speckle.type(torch.float32).cuda()
        
        output = model(phase.unsqueeze(1))
        output = output.squeeze(1)
        
        loss = loss_fn(s * output, speckle)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
    
    print(f'Epoch {i}, Train Loss {np.mean(np.array(train_loss)):.6f}')
    
    model = model.eval()
    test_loss = []
    with torch.no_grad():
        for phase, speckle in test_loader:
            phase = phase.type(torch.float32).cuda()
            speckle = speckle.type(torch.float32).cuda()
        
            output = model(phase.unsqueeze(1))
            output = output.squeeze(1)
            
            loss = loss_fn(s * output, speckle)
            test_loss.append(loss.item())
        
        print(f'Epoch {i}, Test Loss {np.mean(np.array(test_loss)):.6f}')
            

========= Epoch : 0 =========
Epoch 0, Train Loss 0.042338
Epoch 0, Test Loss 0.070009
========= Epoch : 1 =========
Epoch 1, Train Loss 0.042083
Epoch 1, Test Loss 0.073567
========= Epoch : 2 =========
Epoch 2, Train Loss 0.041751
Epoch 2, Test Loss 0.071851
========= Epoch : 3 =========
Epoch 3, Train Loss 0.041275
Epoch 3, Test Loss 0.071387
========= Epoch : 4 =========
Epoch 4, Train Loss 0.041337
Epoch 4, Test Loss 0.071473
========= Epoch : 5 =========
Epoch 5, Train Loss 0.041213
Epoch 5, Test Loss 0.070832
========= Epoch : 6 =========
Epoch 6, Train Loss 0.041060
Epoch 6, Test Loss 0.072967
========= Epoch : 7 =========
Epoch 7, Train Loss 0.040599
Epoch 7, Test Loss 0.073308
========= Epoch : 8 =========
Epoch 8, Train Loss 0.041089
Epoch 8, Test Loss 0.070896
========= Epoch : 9 =========
Epoch 9, Train Loss 0.040260
Epoch 9, Test Loss 0.071965
========= Epoch : 10 =========
Epoch 10, Train Loss 0.040535
Epoch 10, Test Loss 0.070664
========= Epoch : 11 =========
Epoch 11,

KeyboardInterrupt: 

In [7]:
from PIL import Image
import numpy as np
import torch

path = 'apple.png'
image = Image.open(path).convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(N,N))
phase = image / 255 * 2 * np.pi

phase = torch.from_numpy(phase).cuda()
phase = phase.type(torch.float32)
phase = phase.unsqueeze(0).unsqueeze(0)
output = model(phase)

output = output.squeeze(0).squeeze(0)
output = output.detach().cpu().numpy()
output = output / np.max(output) * 255
Image.fromarray(output.astype('uint8')).save('output.png')

In [310]:
image = Image.open('apple.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(224,224))
phase = image / 255 * 2 * np.pi

speckle = create_speckle(phase)
speckle = speckle / np.max(speckle) * 225
Image.fromarray(speckle.astype('uint8')).save(f'speckle.png')

In [347]:
class IterativeInverter(nn.Module):
    def __init__(self, forward_model, N=224):
        super().__init__()
        # 1. 학습된 forward_model에서 고정된 물리 파라미터를 가져옵니다.
        #    이 파라미터들은 더 이상 학습되지 않으므로 buffer로 등록합니다.
        self.register_buffer('diffuser', forward_model.diffuser.detach())
        
        self.N = N
        self.sim_size = 4 * N            # 896
        self.pad_size = (self.sim_size - N) // 2
        self.out_size = 2 * N            # 448
        self.crop_start = (self.sim_size - self.out_size) // 2
        
        # 2. Diffuser와 Pupil을 포함하는 고정된 시스템 함수 H를 미리 계산합니다.
        D = torch.exp(1j * self.diffuser)
        self.register_buffer('H', D)
        
        # 3. H의 역연산에 사용할 켤레 복소수(complex conjugate)를 계산합니다.
        #    H가 0인 부분을 나누면 안되므로, 작은 값(epsilon)을 더해 안정성을 확보합니다.
        epsilon = 1e-8
        self.register_buffer('H_inv', torch.conj(self.H) / (torch.abs(self.H)**2 + epsilon))
        
    def forward_propagation(self, input_phase):
        """ 입력 위상 -> 스페클 이미지 (기존 SpeckleSimulator와 동일) """
        M = torch.exp(1j * input_phase)
        M = F.pad(M, pad=(self.pad_size, self.pad_size, self.pad_size, self.pad_size), 
                               mode='constant', value=0)
        
        Fm = torch.fft.fftshift(torch.fft.fft2(M)) * self.H.cuda()
        U = torch.fft.ifft2(torch.fft.ifftshift(Fm))
        U = torch.flip(U, dims=[-1])
        U = U[..., self.crop_start:self.crop_start+self.out_size,
               self.crop_start:self.crop_start+self.out_size]
        
        U_real = U.real
        U_imag = U.imag
        
        U_real = F.interpolate(U_real.unsqueeze(0).unsqueeze(0), size=(self.N, self.N), mode='area')
        U_imag = F.interpolate(U_imag.unsqueeze(0).unsqueeze(0), size=(self.N, self.N), mode='area')
        U = torch.complex(U_real, U_imag)
        U = U.squeeze(0).squeeze(0)
        return U


    def backward_propagation(self, output_field):
        """ 출력 필드 -> 입력 위상 (물리적 역연산) """
        # U = ifft(fft(M)*H)  ==> M = fft(ifft(U)/H)
        output_field = torch.flip(output_field, dims=[-1])
        U_real = output_field.real
        U_imag = output_field.imag
        U_real = F.interpolate(U_real.unsqueeze(0).unsqueeze(0), size=(2 * self.N, 2 * self.N), mode='area')
        U_imag = F.interpolate(U_imag.unsqueeze(0).unsqueeze(0), size=(2 * self.N, 2 * self.N), mode='area')
        U = torch.complex(U_real, U_imag)
        U = U.squeeze(0).squeeze(0)
        
        # ③ 시뮬레이터에서 잘라냈던 영역을 원본 시뮬레이션 캔버스(896×896)에 재삽입
        U_full = torch.zeros(U.shape[:-2] + (self.sim_size, self.sim_size),
                             dtype=torch.cfloat, device=U.device)
        U_full[..., self.crop_start:self.crop_start+self.out_size,
                self.crop_start:self.crop_start+self.out_size] = U
        
        F_prime = torch.fft.fftshift(torch.fft.fft2(U_full)) * self.H_inv
        M_prime = torch.fft.ifft2(torch.fft.ifftshift(F_prime))
        # ⑤ 앞서 넣었던 zero-padding 제거(224×224)
        M_prime = M_prime[..., self.pad_size:-self.pad_size,
                      self.pad_size:-self.pad_size]
        return M_prime

    def find_phase(self, target_image, iterations=50):
        """ Gerchberg-Saxton 알고리즘으로 최적의 위상을 찾습니다 """
        # 목표 진폭을 계산하고 0으로 나누는 것을 방지
        target_amplitude = torch.sqrt(target_image.clamp(min=1e-8))
        
        # 시작점: 입력 위상을 랜덤하게 초기화
        input_phase = torch.rand_like(target_image) * 2 * torch.pi

        # 반복 최적화 시작
        for _ in range(iterations):
            # 1. 현재 위상으로 출력 필드를 계산 (정방향 전파)
            output_field = self.forward_propagation(input_phase)
            
            # 2. 출력 평면 제약조건 적용:
            #    계산된 위상(torch.angle)은 유지하되, 진폭은 우리가 원하는 목표 진폭으로 강제 교체!
            estimated_phase_at_output = torch.angle(output_field)
            corrected_output_field = target_amplitude * torch.exp(1j * estimated_phase_at_output)
            
            # 3. 수정된 출력 필드를 역방향으로 전파하여 입력 필드를 추정
            estimated_input_field = self.backward_propagation(corrected_output_field)
            
            # 4. 입력 평면 제약조건 적용:
            #    우리의 입력 장치는 위상만 조절 가능하므로, 추정된 입력 필드의 진폭은 버리고 위상만 취함.
            input_phase = torch.angle(estimated_input_field)

        return input_phase.detach() # 최종적으로 찾아낸 위상을 반환

In [352]:
# 학습된 모델을 'forward_model'로 저장
forward_model = model.eval()

# 1. 학습된 forward_model로 Inverter 인스턴스 생성
inverter = IterativeInverter(forward_model).to(device)

# 2. 목표 이미지 준비 (이전과 동일)
target_img = Image.open('square.jpg').convert('L')
target_img = np.array(target_img)
target_ima = 255 - target_img
target_img = cv2.resize(target_img, dsize=(224, 224))
target_img = target_img / np.max(target_img)
target_image = torch.from_numpy(target_img).to(device, dtype=torch.float32)

# 3. Inverter를 사용하여 최적 위상 찾기
print("최적의 위상 패턴을 찾는 중... (반복 최적화)")
# .find_phase() 메서드를 호출하면 내부적으로 루프가 돌면서 위상을 찾아줍니다.
optimized_phase = inverter.find_phase(target_image, iterations=200)
print("완료!")

# 4. 결과 검증: 찾은 위상을 원래의 forward_model에 넣어 사과가 나오는지 확인
with torch.no_grad():
    # 찾은 위상을 정방향 시뮬레이터에 입력
    final_output = forward_model(optimized_phase.unsqueeze(0).unsqueeze(0))
    final_output = final_output.squeeze()


최적의 위상 패턴을 찾는 중... (반복 최적화)
완료!


In [353]:
temp = optimized_phase.detach().cpu().numpy()
temp = temp / np.max(temp) * 255
Image.fromarray(temp.astype('uint8')).save('phase.png')

In [354]:
temp = final_output.detach().cpu().numpy()
temp = temp / np.max(temp) * 255
Image.fromarray(temp.astype('uint8')).save('result.png')